In [1]:
from IPython.display import Markdown as md
import datetime
import os
try:
    os.chdir(os.path.join(os.getcwd(), '../'))
    print(os.getcwd())
except:
    pass

from ia.common.jira.connection import connect
import ia.execution.algo as algo
import ia.execution.conf.components as components
import ia.common.viz.conf.dashboard as conf

report_start_time = datetime.datetime.now()
report_timestamp = f'`Report executed at {report_start_time:%Y-%m-%d %H:%M}`'
md(report_timestamp)

# Access variables

jira_url = os.environ['JIRA_URL']
jira_username = os.environ['JIRA_USERNAME']
jira_password = os.environ['JIRA_PASSWORD']

conf_url = os.environ['CONFLUENCE_URL']
conf_username = os.environ['CONFLUENCE_USERNAME']
conf_password = os.environ['CONFLUENCE_PASSWORD']

# Report variables

space = "~lswolkien"
parent_page = "Reports"
page_title = "Execution report"
projects = {
    'DANCOE': ("COE - Sprint board", "COE Sprint"),
    'DANMR':  ("Retention - Sprint board", "Retention Sprint"),
    'DANSDP': ("SDP Core - Sprint Board", "SDP Core Platform Sprint"),
    'DANTF': ("SDP Tools & Frameworks - Sprint Board", "SDPTF Sprint"),
    'DANMIG': ("SDP Migration - Stories Board", "SDP Migration Sprint"),
    'DANMRGCORE': ("MrGreen Core Board", "MrGreen Core Sprint"),
    "DANUNCO": ("UNO Core - Sprint board", "UNO Sprint"),
    "DANSRE": ("SRE - Sprint Board", "Week")
}

last_sprints = 3

dashboard = conf.Dashboard(conf_url, conf_username, conf_password, page_title, space, parent_page)
jira_access = connect(jira_url, basic_auth=(jira_username, jira_password))

projects_progress = {}

for project_key, _ in projects.items():
    projects_progress[project_key] = algo.active_sprint_progress(jira_access, project_key)


report_head = conf.Component(
    conf.report_head, # callable which returns content and attachments for the dashboard element
    [f"DM execution report in current sprint", "Percentage of work commited to delivered"] # arguments for callable object
)

report_execution = conf.Component(
    components.execution_report,
    [projects_progress]
)

# history of execution

squad_execution_history = []
for project_key, board_sprint in projects.items():
    board_name = board_sprint[0]
    sprint_prefix = board_sprint[1]
    sprints = algo.last_sprints(jira_access, board_name, last_sprints=last_sprints, sprint_name_prefix=sprint_prefix)
    history = algo.progress_history(jira_access, project_key, sprints)
    report_history_header = conf.Component(conf.report_heading,["h2", f"{project_key} execution history"])
    report_execution_history = conf.Component(components.history_execution_report, [history])
    carry_over_issues = conf.Component(components.carry_over_issues, [jira_access, project_key])

    squad_execution_history.append(report_history_header + report_execution_history + carry_over_issues)

# Create or Overwrite Confluance page
dashboard.publish([
        report_head, 
        report_execution, 
        *squad_execution_history
    ])

/Users/lswolkien/Devel/EngineeringMetrics


In [2]:
report_end_time = datetime.datetime.now()
report_timestamp = f'` Report finished at {report_end_time:%Y-%m-%d %H:%M}, time elapsed: {report_end_time - report_start_time}`'
md(report_timestamp)

` Report finished at 2020-06-19 10:03, time elapsed: 0:02:32.816327`

In [3]:
import glob
files = glob.glob('*.png')
for filename in files:
    try:
        os.remove(filename)
    except:
        print("Error while deleting file : ", filename)